# Composing Methods

- Much of refactoring is related to correctly composing methods (i.e. putting stuff together)

- In most cases, excessively long methods are what cause problems
    - Their length conceal execution logic
    - And makes it hard to debug when things go wrong

- Problem
    - You have a code fragment that can be grouped

- Solution
    - Group it in a method, and replace the original code with a method call

- Motivation
    - Minimise the number of lines in a method, which makes it easier to figure out what it does

- How to refactor
    - Make a new method and name it self-evidently
    - Copy the code fragment to your new method, and delete the existing fragment, calling the new method instead

- Relationships with other refactoring methods
    - Opposite 
        - Inline Method
    - Similar
        - Move Method
    - Related
        - Introduce Parameter Object
        - Form Template Method
        - Parameterize Method

- Related code smells
    - Duplicate Code
    - Long Method
    - Feature Envy
    - Switch Statements
    - Message Chains
    - Comments
    - Data Class

- Example:

## 1. Extract Method

- Problem
    - You have a code fragment that can be grouped

- Solution
    - Group it in a method, and replace the original code with a method call

- Motivation
    - Minimise the number of lines in a method, which makes it easier to figure out what it does

- How to refactor
    - Make a new method and name it self-evidently
    - Copy the code fragment to your new method, and delete the existing fragment, calling the new method instead

- Relationships with other refactoring methods
    - Opposite 
        - Inline Method
    - Similar
        - Move Method
    - Related
        - Introduce Parameter Object
        - Form Template Method
        - Parameterize Method

- Related code smells
    - Duplicate Code
    - Long Method
    - Feature Envy
    - Switch Statements
    - Message Chains
    - Comments
    - Data Class

- Example:

In [2]:
from dataclasses import dataclass
from abc import abstractmethod

@dataclass
class Borrower:
    name: str
    amount_outstanding: float
    address: str

    def bad__introduce_yourself(self):
        print(self.name)
        print(self.amount_outstanding)
        print(self.address)

    def good__introduce_yourself(self):
        self._print_all_details()

    def _print_all_details(self):
        print(self.name)
        print(self.amount_outstanding)
        print(self.address)

test = Borrower(name='testname', amount_outstanding=123, address='my_address')
test.good__introduce_yourself()

testname
123
my_address


## 2. Inline Method

- Problem
    - A method body is more obvious than the method

- Solution
    - Move the method's logic into the caller, and delete the method

- Motivation
    - Sometimes, you end up separating stuff out when it's actually trivial, and creating a method adds unnecessary bloat
    - In such cases, it can be clearer to just leave stuff in the original caller

- How to refactor
    - Ensure the method isn't defined/used in subclasses
    - Find all calls to this method, and replace with the method's logic
    - Delete method

- Relationships with other refactoring methods
    - Opposite 
        - Extract Method

- Related code smells
    - Speculative Generality

- Example:

In [ ]:
class CabDriver:
    def __init__(self, name: str):
        self.name: str = name
    
    def bad__has_long_name(self):
        return self._name_is_long()
    
    def _name_is_long(self):
        return len(self.name.split(' ')) >= 3
    
    def good__has_long_name(self):
        return len(self.name.split(' ')) >= 3

## 3. Extract Variable

- Problem
    - An expression is hard to understand (multiple if condition, complex switch case, etc) 

- Solution
    - Put the result of the expressino or parts into separate variables that are self-explanatory

- Motivation
    - Helps with understanding what you want to achieve, which may otherwise necessitate comments etc
    - BUT it makes your code longer
    - Also be aware of performance tradeoffs
        - If you call `if a or b`, once `a` is true, you will not compute `b`
        - However, if you extract the variable out, you will always compute both `a` and `b`

- How to refactor
    - Assign a new variable for each of the items you want to abstract

- Relationships with other refactoring methods
    - Opposite 
        - Inline Temp
    - Similar
        - Extract Method

- Related code smells
    - Comments

- Example:

In [ ]:
class Borrower:
    def __init__(self, age: int, income: float):
        self.age: int = age
        self.income: float = income
    
    def bad__decide_eligible(self) -> bool:
        if (self.age >= 65) and (self.income <= 1000):
            return False
        return True
    
    def good__decide_eligible(self) -> bool:
        is_too_old: bool = self.age >= 65
        is_earning_too_little: bool = self.income <= 1000
        if is_too_old and is_earning_too_little:
            return False
        return True

## 4. Inline Temp

- Problem
    - You have a temporary variable that's assigned the result of some simple expression

- Solution
    - Replace the reference to the variable with the expression itself

- Motivation
    - This is the tradeoff with `Extract Variable` above
    - If you extract too much, you end up with ridiculously longer code for no good reason

- Drawback
    - Temp variables are sometimes used as a caching mechanism, so check that there is no performance implication before switching strategies

- How to refactor
    - Delete variable and leave logical evaluation to where it is used

- Relationships with other refactoring methods
    - Related
        - Replace Temp with Query
        - Extract Method

- Example:

In [3]:
class Borrower:
    def __init__(self, age: int, income: float):
        self.age: int = age
        self.income: float = income
    
    def good__decide_eligible(self) -> bool:
        if (self.age >= 65) and (self.income <= 1000):
            return False
        return True
    
    def bad__decide_eligible(self) -> bool:
        is_too_old: bool = self.age >= 65
        is_earning_too_little: bool = self.income <= 1000
        if is_too_old and is_earning_too_little:
            return False
        return True

## 5. Replace Temp with Query

- Problem
    - You have the result of an expression stored as a local variable for later use in your code

- Solution
    - Move the whole expression to a separate method and return the result from the method instead

- Motivation
    - If this expression is commonly used, having it as a proper method can promote re-use

- Drawback
    - This will likely create a performance dip, albeit minimal, because you are querying a new method

- How to refactor
    - Move expression to a method and call method instead of expression

- Relationships with other refactoring methods
    - Similar
        - Extract Method

- Related code smells
    - Long Method
    - Duplicate Code
    
- Example:

In [4]:
class Borrower:
    def __init__(self, age: int, income: float, nric_number: int):
        self.age: int = age
        self.income: float = income
        self.nric_number: int = nric_number
    
    def _is_earning_too_little(self) -> bool:
        income_cutoff = (self.nric_number // self.age) * self.income
        if self.income < income_cutoff:
            return True
        return False
    
    def bad__decide_eligible(self) -> bool:
        is_too_old: bool = self.age >= 65
        is_earning_too_little: bool = self.income <= (self.nric_number // self.age) * self.income
        if is_too_old and is_earning_too_little:
            return False
        return True
    
    def good__decide_eligible(self) -> bool:
        is_too_old: bool = self.age >= 65
        if is_too_old and self._is_earning_too_little():
            return False
        return True

## 6. Split Temporary Variable

- Problem
    - You have a local variable (e.g. `temp`) that stores various unrelated intermediate value in a method

- Solution
    - Use different variables for different values

- Motivation
    - If anything goes wrong in your computation, `temp` can end up passing the wrong value

- How to refactor
    - Make multiple new variables and name them in an informative way

- Relationships with other refactoring methods
    - Opposite 
        - Inline temp
    - Similar
        - Extract Variable
        - Remove Assignments to Parameters
    - Related
        - Extract Method

- Example:

In [ ]:
class Square:
    def __init__(self, length: int):
        self.length: int = length
    
    def bad__print_perimeter_and_area(self):
        temp = self.length*4
        print(f'perimeter = {temp}')

        temp = self.length**2
        print(f'area = {temp}')
    
    def good__print_perimeter_and_area(self):
        perimeter = self.length*4
        print(f'{perimeter=}')

        area = self.length**2
        print(f'{area=}')

## 7. Remove Assignments to Parameters

- Problem
    - Some value assigned to parameter in a method's body

- Solution
    - Use a local variable instead of a parameter

- Motivation
    - Error prone. If your input parameter can change depending on what happens in the function, you won't know what value it is taking at any point in time

- How to refactor
    - Make a new local variable and assign value
    - Call the local variable instead

- Relationships with other refactoring methods
    - Similar
        - Split Temporary Variable
    - Related
        - Extract Method
        
- Example:

In [ ]:
def bad__discount(price, quantity):
    if quantity > 50:
        price -= 2
    return price

def good__discount(price, quantity):
    new_price = price
    if quantity > 50:
        new_price -= 2
    return new_price


## 8. Replace method with method object

- Problem
    - You have a long method where the local variables are so spaghettied together that you can't extract part of it as another method to simplify it

- Solution
    - Separate the method out to its own class, and delegate the original function to the new class

- Motivation
    - This helps you isolate the problem to a single class, and turn all local variables into class fields instead, which allows for clarity
    - By making things clearer, it then lets you systematically reduce the method into constituent components

- How to refactor
    - Create a new class, named after the purpose of the method you are refactoring
    - If some data needed from the original class, create private field to store a reference to an instance of the original class where the method came from 
    - Create private fields for each local variable used in the method
    - Create constructor that accepts the values of these local variables as parameters
    - Create main() method with the code from method you want to copy over
    - Replace the original method by creating a method object and calling main

- Relationships with other refactoring methods
    - Similar
        - Replace data value with object

- Related code smells
    - Long Method

- Example:

In [ ]:
class bad__Order:
    def get_price(self) -> float:
        primaryBasePrice = 1
        price_adjustment_a = 2
        price_adjustment_b = 3
        
        ## Some long computation
        ...

###

class good__Order:
    def get_price(self):
        return PriceGetter(self).get_price()

class PriceGetter:
    def __init__(self, order: good__Order):
        self._primaryBasePrice = 1
        self._price_adjustment_a = 2
        self._price_adjustment_b = 3

    def get_price(self) -> float:
        ...


## 9. Substitute Algorithm

- Problem
    - You have an algorithm (method) that you want to replace with a new one

- Solution
    - Replace the body of the method you want to change with the new algorithm, while keeping everything else the same (i.e. same method name/signature)

- Motivation
    - Sometimes, it is easier to start from scratch when things grow too complicated. This is basically saying, I tear down everything and re-implement

- How to refactor
    - Simplify the existing algorithm as much as possible
    - Create new algorithm in a new method
    - Replace old with new method and test
    - If results don't match, investigate and fi
    - Once all tests pass, cutover for good

- Related code smells
    - Duplicate Code
    - Long Method

- Example:

In [ ]:
def _return_young_primes(arr: list[int]) -> list[int]:
    young_primes = []
    if 2 in arr:
        young_primes.append(2)
    if 3 in arr:
        young_primes.append(3)
    if 5 in arr:
        young_primes.append(5)
    return young_primes

def return_young_primes(arr: list[int]) -> list[int]:
    young_primes = [x for x in arr if x in [2,3,5]]
    return young_primes